In [24]:
import pandas as pd
from pathlib import Path
from constants import NOMINAL_COLUMNS, DISCRETE_COLUMNS, CONTINOUS_COLUMNS, TEXT_COLUMNS, BINARY_COLUMNS
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from MLModel import MLModel
import utils
import pickle

In [25]:
model_handler = MLModel()

fill_values_nominal = model_handler.load_model('artifacts/nan_outlier_handler/fill_values_nominal.pkl')
fill_values_discrete = model_handler.load_model('artifacts/nan_outlier_handler/fill_values_discrete.pkl')
fill_values_continuous = model_handler.load_model('C:\\Users\\david\\Programming\\MLOps\\4th_Homework\\artifacts\\nan_outlier_handler\\fill_values_continuous.pkl')
min_max_scaler_dict = model_handler.load_model('artifacts/encoders/min_max_scaler_dict.pkl')
onehot_encoders = model_handler.load_model('artifacts/encoders/onehot_encoders_dict.pkl')
xgb = model_handler.load_model('artifacts/models/xgb_model.pkl')

In [26]:
data_path = 'temp_train.csv'
df = pd.read_csv(data_path)

In [27]:
sample_data = df.iloc[0]
sample_data = pd.DataFrame([sample_data])

In [28]:
sample_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [9]:
print(sample_data.index)

Index([0], dtype='int64')


In [29]:
sample_data = sample_data.replace('?', np.nan)

sample_data = utils.extract_features(sample_data, "Ticket")

sample_data.head()

for col in CONTINOUS_COLUMNS:  
    sample_data[col] = pd.to_numeric(sample_data[col], errors='coerce')

for col in NOMINAL_COLUMNS:
    sample_data[col].fillna(fill_values_nominal[col], inplace=True)

for col in DISCRETE_COLUMNS:
    sample_data[col].fillna(fill_values_discrete[col], inplace=True)

sample_data[DISCRETE_COLUMNS] = sample_data[DISCRETE_COLUMNS].astype(int)

for col in CONTINOUS_COLUMNS:
    sample_data[col].fillna(fill_values_continuous[col], inplace=True)

sample_data.drop(columns=TEXT_COLUMNS, inplace=True)

sample_data[BINARY_COLUMNS] = sample_data[BINARY_COLUMNS].astype(int)

for col, encoder in onehot_encoders.items():
    new_data = encoder.transform(sample_data[col].to_numpy().reshape(-1, 1))
    new_df = pd.DataFrame(new_data, columns=encoder.get_feature_names_out([col]))
    sample_data = pd.concat([sample_data, new_df], axis=1).drop(columns=[col])

for col, scaler in min_max_scaler_dict.items():
    expected_features = scaler.feature_names_in_ if hasattr(scaler, "feature_names_in_") else [col]

    # Ellenőrizzük, hogy minden elvárt oszlop benne van
    missing = [f for f in expected_features if f not in sample_data.columns]
    if missing:
        print(f"Skipping scaler for {col}: missing columns {missing}")
        continue

    # Skálázás
    scaled = scaler.transform(sample_data[expected_features])
    sample_data[expected_features] = scaled


if 'Survived' in sample_data.columns:
    sample_data = sample_data.drop(columns=['Survived'])

predicted_value = xgb.predict(sample_data)

print(f"The predicted value is: {predicted_value[0]}")

C:\Users\david\AppData\Local\Temp\ipykernel_9028\1827607092.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'B96 B98' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sample_data[col].fillna(fill_values_nominal[col], inplace=True)


The predicted value is: 0


In [31]:
import pandas as pd

inference_json = {
    "inference_row": [
        {
            "PassengerId": "1",
            "Survived": "0",
            "Pclass": "3",
            "Name": "Braund, Mr. Owen Harris",
            "Sex": "male",
            "Age": "22.0",
            "SibSp": "1",
            "Parch": "0",
            "Ticket": "A/5 21171",
            "Fare": "7.25",
            "Cabin": "NaN",  
            "Embarked": "S"
        }
    ]
}

smth = inference_json["inference_row"]

smth2 = pd.DataFrame(smth)
smth2.head()

# Convert discrete columns to numeric first, then to int
for col in DISCRETE_COLUMNS:
    smth2[col] = pd.to_numeric(smth2[col], errors='coerce').astype(int)